In [ ]:
#####################################################################
#
#   aiplatform.CustomContainerTrainingJob
#
#####################################################################

# Create the training file/s and the dockerfile

In [ ]:
# create directory
! mkdir -p devdir/trainer

In [ ]:
#-------------------
# task.py
#-------------------

In [ ]:
%%writefile devdir/trainer/task.py

import os
import argparse
from google.cloud import storage

# parse args
parser = argparse.ArgumentParser()
parser.add_argument('--bucket_name', dest='bucket_name', default="", type=str, help='The GCS bucket to store model artifacts -> w/o gs://')
parser.add_argument('--train_ds', dest='train_ds', default="", type=str, help='file name of the training data')
args = parser.parse_args()

## set params
DATA_BUCKET_NAME = args.bucket_name
DATA_BUCKET_ROOT = f"/gcs/{DATA_BUCKET_NAME}"

DS_TRAIN_FILENAME = args.train_ds
DS_TRAIN_PATH = f"{DATA_BUCKET_ROOT}/{DS_TRAIN_FILENAME}"

MODEL_DIR = os.environ["AIP_MODEL_DIR"]
JOB_TYPE = os.environ["JOB_TYPE"]

# Upload the saved model file to GCS -> using the client library
my_filename = f"info_{JOB_TYPE}.txt"
my_filename = f"{DATA_BUCKET_ROOT}/info_{JOB_TYPE}.txt"
with open(my_filename, 'w') as f:
    f.write( DATA_BUCKET_NAME + "\n")
    f.write( DATA_BUCKET_ROOT + "\n")
    f.write( DS_TRAIN_FILENAME + "\n")
    f.write( DS_TRAIN_PATH + "\n")
    f.write( MODEL_DIR + "\n")
    f.write( JOB_TYPE + "\n")
    f.write( "\n")
    f.write('-'*30)
    f.write( "\n")
    f.write( "ENV_VARS" + "\n")
    for k, v in sorted(os.environ.items(), key=lambda x:x[0]):
        f.write(f"{k}={v}" + "\n")    
    f.write("\n")
    f.write('-'*30)
    f.write("\n")
    f.write("TOP_LEVEL_LISTDIR")
    f.write("\n")
    for i in os.listdir():
        f.write(i+"\n")        
        
#storage_client = storage.Client()
#storage_path = os.path.join(MODEL_DIR, my_filename)
#blob = storage.blob.Blob.from_string(storage_path, client=storage_client)
#blob.upload_from_filename(my_filename)

In [ ]:
#-------------------
# Dockerfile
#-------------------

In [ ]:
%%writefile devdir/Dockerfile

# Specifies base image and tag
FROM gcr.io/deeplearning-platform-release/tf2-gpu.2-8

WORKDIR /

# Installs additional packages
RUN pip install pip --upgrade

# Copies the trainer code to the docker image.
COPY trainer /trainer

# Sets up the entry point to invoke the trainer.
ENTRYPOINT ["python", "-m", "trainer.task"]

# Build the docker file and push to artifact registry

In [ ]:
from datetime import datetime
import os

In [ ]:
# specify parameters
P = ! gcloud config list --format 'value(core.project)'
PROJECT_ID = P[0]
P = ! gcloud projects list --filter="$(gcloud config get-value project)" --format="value(PROJECT_NUMBER)"
PROJECT_NUMBER = P[0]
REGION = "us-central1"
SERVICE_ACCOUNT = f"sa-vertex-pipelines@{PROJECT_ID}.iam.gserviceaccount.com"

# exercise details
USE_CASE = "CustomContainerTrainingJob"
ML_FRAMEWORK = "scikit"
MODEL_TYPE = "binclass"
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

# repo details
REPO_NAME = f"{USE_CASE}"
REPO_FORMAT = "docker"
REPO_DESCRIPTION = f"'Description for {REPO_NAME}'"
IMAGE_URI = f"{REGION}-{REPO_FORMAT}.pkg.dev/{PROJECT_ID}/{REPO_NAME}/{USE_CASE}:latest"


# GCS inputs
GCS_BUCKET_NAME = f"bkt-{REGION}-{USE_CASE}"
GCS_BUCKET_PATH = f"gs://{GCS_BUCKET_NAME}"
GCS_BUCKET_PATH_DATA = f"{GCS_BUCKET_PATH}/data"
GCS_BUCKET_PATH_CONFIGS = f"{GCS_BUCKET_PATH}/configs"
GCS_BUCKET_PATH_TMP = f"{GCS_BUCKET_PATH}/tmp"
GCS_BUCKET_PATH_STAGING = f"{GCS_BUCKET_PATH}/staging"

TRAIN_DS = "tab_class_10inps_1krows_tes_3498.csv"

In [ ]:
# create the repo and configure
! gcloud artifacts repositories create {REPO_NAME} --repository-format={REPO_FORMAT} --location={REGION} --description={REPO_DESCRIPTION}
! gcloud auth configure-{REPO_FORMAT} {REGION}-{REPO_FORMAT}.pkg.dev

In [ ]:
# build and push
! docker build ./devdir/ -t {IMAGE_URI}
! docker push {IMAGE_URI}

# Kick off CustomContainerTrainingJob

In [ ]:
from google.cloud import aiplatform

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION)

job = aiplatform.CustomContainerTrainingJob(
    display_name = f"CustomContainerTrainingJob_{TIMESTAMP}"
    , project = PROJECT_ID
    , location = REGION
    , staging_bucket = GCS_BUCKET_PATH
    , container_uri = IMAGE_URI
)

In [ ]:
job.run(service_account = SERVICE_ACCOUNT
        #, network = f"projects/{PROJECT_NUMBER}/global/networks/{NETWORK}"
        , replica_count = 1
        , machine_type = "n1-standard-4"
        #, model_display_name = "mymodelname"
        #, model_labels = { 'MY_KEY': 'MY_VALUE' }
        , args = [f"--bucket_name={GCS_BUCKET_NAME}", f"--train_ds={TRAIN_DS}"]
        , environment_variables = { 'JOB_TYPE': 'CustomContainerTrainingJob' }
       )